In [202]:
# Prepare environment and load data
import pandas as pd
import nltk
import feather
import re, math
from collections import Counter

event_dict = feather.read_dataframe('../parsed_data/parsed_dict.feather')

In [203]:
## Prepare event dataframe

# re.sub(r'(^|\s)x(\w+,)', r'', x)

event_dict.drop(['NA'], inplace=True, axis=1)
event_dict = event_dict.transpose()
event_dict.columns = event_dict.loc['event']
event_dict = event_dict.reindex(event_dict.index.drop(['event']))
event_dict = event_dict.applymap(lambda z: re.sub(r'(^|\s)x(\w+,)', r'', z))
event_dict.head()

event,benghazi,cancer,wiki_leak,doctrine,arab_spring,russian_reset
dictionary,"benghazi, attack, took, place, even, septemb, ...","cancer, group, diseas, involv, abnorm, cell, g...","unit, state, diplomat, cabl, leak, wide, known...","hillari, doctrin, term, us, describ, agenda, f...","arab, spring, arab, ar, rab, al, revolutionari...","russian, reset, attempt, obama, administr, imp..."


In [204]:
event_dict1 = event_dict.T
event_dict2 = event_dict

# Helper function to convert text to vector
WORD = re.compile(r'\w+')

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

# Helper function to compute cosine similarity
def cosine_sim(vect1, vect2):
    a = set(vect1)
    b = set(vect2)
    top = len(a & b)
    bottom = len(a | b)
    if not bottom:
        sim = 0
    else:
        sim = top / float(bottom)

    return sim

In [205]:
# Create new dataframe to contain cosine similarities
# cosim_df = pd.DataFrame.from_dict(dict([for l in list_events ]))
cosim_df = pd.DataFrame()

beng = text_to_vector(event_dict.benghazi[0])
canc = text_to_vector(event_dict.cancer[0])
wiki = text_to_vector(event_dict.wiki_leak[0])
doct = text_to_vector(event_dict.doctrine[0])
spring = text_to_vector(event_dict.arab_spring[0])
russ = text_to_vector(event_dict.russian_reset[0])
# print cosine_sim(beng, hill)

benghazi = []
cancer = []
wiki_leak = []
doctrine = []
arab_spring = []
russian_reset = []

event_dict1 = event_dict.applymap(text_to_vector)

for index, vocabulary in event_dict1.iterrows():
    for vocab in vocabulary:
        benghazi.append(round(cosine_sim(beng, vocab), 3))
        cancer.append(round(cosine_sim(canc, vocab), 3))
        wiki_leak.append(round(cosine_sim(wiki, vocab), 3))
        doctrine.append(round(cosine_sim(doct, vocab), 3))
        arab_spring.append(round(cosine_sim(spring, vocab), 3))
        russian_reset.append(round(cosine_sim(russ, vocab), 3))
        
        
cosim_df['benghazi'] = benghazi
cosim_df['cancer'] = cancer
cosim_df['wiki_leak'] = wiki_leak
cosim_df['doctrine'] = doctrine
cosim_df['arab_spring'] = arab_spring
cosim_df['russian_reset'] = russian_reset

In [206]:
cosim_df1 = cosim_df.copy()
cosim_df1.index = list(cosim_df.columns.values)
cosim_df1

,benghazi,cancer,wiki_leak,doctrine,arab_spring,russian_reset
benghazi,1.000,0.182,0.240,0.147,0.272,0.068
cancer,0.182,1.000,0.156,0.130,0.173,0.046
wiki_leak,0.240,0.156,1.000,0.159,0.226,0.080
doctrine,0.147,0.130,0.159,1.000,0.152,0.080
arab_spring,0.272,0.173,0.226,0.152,1.000,0.063
russian_reset,0.068,0.046,0.080,0.080,0.063,1.000


In [159]:
# event_dict.applymap(lambda x: filter(lambda z:z[0]!='x', x.split()))

In [109]:
pd.__version__

u'0.18.1'